In [1]:
import os
os.environ['SPARK_HOME'] = r'C:\Users\Marcos\Documents\Spark'
os.environ['PYSPARK_DRIVER_PYTHON'] = 'jupyter'
os.environ['PYSPARK_DRIVER_PYTHON_OPTS'] = 'lab'
os.environ['PYSPARK_PYTHON'] = 'python'

In [2]:
from pyspark.sql import SparkSession

Q1: Read data with ~| delimiter

In [3]:
spark = SparkSession.builder.appName('DataLost').getOrCreate()

In [11]:
df = spark.read.option('delimiter','~|').csv('data/lost.csv', header=True, inferSchema=True, )

In [18]:
df.show()

+------+---+
|  Name|Age|
+------+---+
|  Jack| 42|
|  Kate| 35|
|Sawyer| 40|
|Hurley| 30|
|  John| 60|
+------+---+



In [38]:
spark.stop()

Q2: Merge 2 datasets

In [39]:
spark = SparkSession.builder.appName('MergeData').getOrCreate()

In [41]:
path_lost = 'data/lost.csv'
path_house = 'data/house.csv'
df_lost = spark.read.option('delimiter','~|').csv(path_lost, header=True, inferSchema=True)
df_house = spark.read.option('delimiter',',,').csv(path_house, header=True, inferSchema=True)

In [42]:
df_lost.createOrReplaceTempView('TableLost')
df_house.createOrReplaceTempView('TableHouse')

In [44]:
df = spark.sql("""
               SELECT * FROM TableLost 
               UNION
               SELECT * FROM TableHouse
               """)
df.show()

+------+---+
|  Name|Age|
+------+---+
|  John| 60|
|Sawyer| 40|
|  Jack| 42|
|  Kate| 35|
|Hurley| 30|
| Cuddy| 45|
| House| 55|
|    13| 27|
| Chase| 33|
|Wilson| 48|
+------+---+



In [43]:
df = df_lost.union(df_house)
df.show()

+------+---+
|  Name|Age|
+------+---+
|  Jack| 42|
|  Kate| 35|
|Sawyer| 40|
|Hurley| 30|
|  John| 60|
| House| 55|
|Wilson| 48|
| Cuddy| 45|
|    13| 27|
| Chase| 33|
+------+---+



In [49]:
df = df_lost.join(df_house, on=['Name','Age'], how='outer')
df.show()

+------+---+
|  Name|Age|
+------+---+
|    13| 27|
| Chase| 33|
| Cuddy| 45|
| House| 55|
|Hurley| 30|
|  Jack| 42|
|  John| 60|
|  Kate| 35|
|Sawyer| 40|
|Wilson| 48|
+------+---+

